In [103]:
import pandas as pd
import numpy as np
# 用於二元目標變數
from scipy.stats import pointbiserialr
from functools import reduce
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve

In [125]:
# Import GDP Data
from data import OverallGeneralEcon, df, MacroInfo, JuridicalInfo
from module1 import predict_for_month, find_best_threshold, predicted_labels, predicted_accuracy

In [5]:
# 名目、實質、時間
"""
MacroInfo
名目有效匯率指數–台灣
台灣消費者信心指數
台灣消費者信心指數–未來半年國內經濟景氣
台灣消費者信心指數–未來半年國內物價水準
美國消費者信心指數
經濟成長率(GDP)–全年
--
實質國內生產毛額(GDP)–台幣 -> 只更新到202009
實質GDP成長率(台灣) -> 只更新到202212
--
貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均
臺灣製造業PMI - 新增出口訂單 - 基礎原物料
台灣外銷訂單金額–全球
消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數
美國核心PPI、美國核心PPI月增率
進口物價指數、出口物價指數年增率

sysallunTWESPF
外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)
景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)

sysTaiStaValMAC
景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)

其他
財報、營收
"""

'\nMacroInfo\n名目有效匯率指數–台灣\n台灣消費者信心指數\n台灣消費者信心指數–未來半年國內經濟景氣\n台灣消費者信心指數–未來半年國內物價水準\n美國消費者信心指數\n經濟成長率(GDP)–全年\n--\n實質國內生產毛額(GDP)–台幣 -> 只更新到202009\n實質GDP成長率(台灣) -> 只更新到202212\n--\n貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均\n臺灣製造業PMI - 新增出口訂單 - 基礎原物料\n台灣外銷訂單金額–全球\n消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數\n美國核心PPI、美國核心PPI月增率\n進口物價指數、出口物價指數年增率\n\nsysallunTWESPF\n外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)\n景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)\n\nsysTaiStaValMAC\n景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)\n\n其他\n財報、營收\n'

#### *【步驟：資料處理 → 分割 train、test dataset → model fit  → 計算出機率&閾值 → 計算準確率】

### 總經資料處理

In [6]:
# Improt data
# 名目有效匯率指數
NominalExchageRate = MacroInfo[MacroInfo['名稱']=='名目有效匯率指數–台灣'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(NominalExchageRate.head(3))

# 台灣消費者信心指數
TW_ICS = MacroInfo[MacroInfo['名稱']=='台灣消費者信心指數'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_ICS.tail(3))

# 消費者物價指數(CPI)年增率
TW_CPIGrowth = MacroInfo[MacroInfo['名稱']=='消費者物價指數(CPI)年增率'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_CPIGrowth.head(3))

# 經濟成長率(GDP)–全年
TW_GDPGrowth = MacroInfo[MacroInfo['名稱']=='經濟成長率(GDP)–單季'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_GDPGrowth.head(3))

,ddate,value,category
1879,202311,101.05,名目有效匯率指數–台灣
3095,202310,100.99,名目有效匯率指數–台灣
4896,202309,101.07,名目有效匯率指數–台灣


,ddate,value,category
258660,201403,80.96,台灣消費者信心指數
260975,201402,82.93,台灣消費者信心指數
263270,201401,80.83,台灣消費者信心指數


,ddate,value,category
309,202312,2.71,消費者物價指數(CPI)年增率
1199,202311,2.90,消費者物價指數(CPI)年增率
2355,202310,3.05,消費者物價指數(CPI)年增率


,ddate,value,category
0,202412,1.06,經濟成長率(GDP)–單季
27,202409,2.12,經濟成長率(GDP)–單季
42,202406,4.77,經濟成長率(GDP)–單季


In [7]:
#JuridicalInfo.columns
#上市買賣超金額(百萬)、上櫃買賣超金額(百萬)、合計買進金額(百萬)

In [8]:
TW_NominalGDP = OverallGeneralEcon[['time','Nominal_GDP']]
TW_NominalGDP = TW_NominalGDP.rename(columns={'time':'ddate'})
#print(TW_NominalGDP.head(3))
TW_NominalGDP['ddate'] = TW_NominalGDP['ddate'].astype(str)

In [9]:
# 將GDP季資料先合併
GDP_quarterly = pd.merge(TW_NominalGDP, TW_GDPGrowth, on='ddate', how='left')
GDP_quarterly.columns = ['ddate', 'NominalGDP', 'GDP_growth', 'category_GDPgrowth']
#GDP_quarterly.head(3)

In [10]:
# (月)總經資料
dfs = [NominalExchageRate, TW_ICS, TW_CPIGrowth]
MergedMacroData = reduce(lambda left, right: pd.merge(left, right, on='ddate', how='left'), dfs)

# Rename columns
MergedMacroData.columns = ['ddate', 'FX', 'category_FX', 'CCI',
                          'category_CCI', 'CPI_growth', 'category_CPIgrowth']

#display(MergedMacroData)
# FX-Foriegn-exchange rate
# CCI-台灣消費者信心指數

In [11]:
MergedMacroData = pd.merge(MergedMacroData, GDP_quarterly, on='ddate', how='left')

In [12]:
MergedMacroData['NominalGDP'] = MergedMacroData['NominalGDP'].ffill()
MergedMacroData['GDP_growth'] = MergedMacroData['GDP_growth'].ffill()
MergedMacroData['category_GDPgrowth'] = MergedMacroData['category_GDPgrowth'].ffill()

In [13]:
MergedMacroData.head(9)

,ddate,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202311,101.05,名目有效匯率指數–台灣,69.40,台灣消費者信心指數,2.90,消費者物價指數(CPI)年增率,NaN,NaN,NaN
1,202310,100.99,名目有效匯率指數–台灣,69.06,台灣消費者信心指數,3.05,消費者物價指數(CPI)年增率,NaN,NaN,NaN
2,202309,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
3,202308,100.67,名目有效匯率指數–台灣,67.51,台灣消費者信心指數,2.53,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202307,101.31,名目有效匯率指數–台灣,68.39,台灣消費者信心指數,1.88,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
5,202306,102.80,名目有效匯率指數–台灣,66.66,台灣消費者信心指數,1.75,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
6,202305,101.84,名目有效匯率指數–台灣,63.67,台灣消費者信心指數,2.02,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
7,202304,101.35,名目有效匯率指數–台灣,63.18,台灣消費者信心指數,2.35,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
8,202303,101.85,名目有效匯率指數–台灣,64.47,台灣消費者信心指數,2.35,消費者物價指數(CPI)年增率,5482976.0,-3.31,經濟成長率(GDP)–單季


### 單純以總經變數訓練模型，準確率約51%

In [38]:
# Rename the column
df = df.rename(columns={'direction':'real_direction'})

In [15]:
# 合併df以及總經資料
TheWhole = pd.merge(df[['ddate', 'stockid', 'real_direction']].astype({'ddate':'str'}), 
                    MergedMacroData, on='ddate', how='left')
#TheWhole[TheWhole['ddate']=='202302'].head(3)

In [16]:
#TheWhole.columns
TheWhole = TheWhole.dropna()
#TheWhole.head()

In [17]:
# Split data by TimeSeriesSplit
tscv_macro = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_macro = list(tscv_macro.split(TheWhole))

In [18]:
# 使用最後一組的 trained model index 切分 training & testing data
train_idx_macro = splits_macro[-1][0]
test_idx_macro = splits_macro[-1][1]

In [19]:
TargetCol = ['real_direction']
VarialbleCol = ['FX', 'CCI', 'CPI_growth',
                'NominalGDP', 'GDP_growth']

# Prepare data for separating
y_macro = TheWhole[TargetCol]
X_macro = TheWhole[VarialbleCol]

In [20]:
# Extract test and train data from overall data
X_train_macro, X_test_macro = X_macro.iloc[train_idx_macro], X_macro.iloc[test_idx_macro]
y_train_macro, y_test_macro = y_macro.iloc[train_idx_macro], y_macro.iloc[test_idx_macro]

In [21]:
# Train model based on the training data
model_macro = LogisticRegression()
model_fit_macro = model_macro.fit(X_train_macro, y_train_macro)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
# Make predictions on the test data
# Calculate thresholds to determine the predicted results
# probability
# display the results(0/1)

In [225]:
# 預測結果為1的機率
prob_macro = predict_for_month(model_fit_macro, TheWhole, VarialbleCol)
threshold_macro = find_best_threshold(TheWhole['real_direction'], prob_macro)
#print(round(threshold_macro, 2)) #0.51

print(round(threshold_macro, 4))

0.5102


In [118]:
# 以閾值為界判斷預測值應為0或1
pred_macro = predicted_labels(prob_macro, threshold_macro)

In [126]:
# 計算總經數值正確性
predicted_accuracy(TheWhole['real_direction'], pred_macro) #0.51

0.51

### 總經指數+營收 訓練+預測

In [184]:
TheWhole['ddate'] = TheWhole['ddate'].astype(str)
df['ddate'] = df['ddate'].astype(str)

In [213]:
df = df.rename({'direction':'real_direction'}, axis='columns')
TheWhole_rev = pd.merge(df, TheWhole, on=('ddate', 'stockid', 'real_direction'), how='inner')
TheWhole_rev.head(5)

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,mond12,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202309,1101,0,8735157.0,-6.87,-16.05,9379408.0,-12.26,4.76,8953462.0,...,-2.67,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
1,202309,1102,1,6691905.0,3.43,-4.32,6469836.0,-8.48,-4.72,6790125.0,...,-1.06,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
2,202309,1103,0,232979.0,-2.49,13.50,238917.0,31.01,-10.39,266631.0,...,12.55,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
3,202309,1104,1,618592.0,0.23,3.59,617197.0,1.93,-3.15,637246.0,...,-1.38,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202309,1108,0,404171.0,-6.10,16.60,430420.0,10.35,-2.43,441143.0,...,-11.13,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季


In [215]:
Varialble_MacroRev = ['revd1', 'yoyd1',
                         'mond1', 'revd2', 'yoyd2', 'mond2', 'revd3', 'yoyd3', 'mond3', 'revd4',
                         'yoyd4', 'mond4', 'revd5', 'yoyd5', 'mond5', 'revd6', 'yoyd6', 'mond6',
                         'revd7', 'yoyd7', 'mond7', 'revd8', 'yoyd8', 'mond8', 'revd9', 'yoyd9',
                         'mond9', 'revd10', 'yoyd10', 'mond10', 'revd11', 'yoyd11', 'mond11',
                         'revd12', 'yoyd12', 'mond12', 'FX', 'CCI', 'CPI_growth',
                         'NominalGDP', 'GDP_growth']

Target_MacroRev = ['real_direction']

In [216]:
# Prepare data for separating
y_MacroRev = TheWhole_rev[Target_MacroRev]
X_MacroRev = TheWhole_rev[Varialble_MacroRev]

# Extract test and train data from overall data
X_train_MacroRev, X_test_MacroRev = X_MacroRev.iloc[train_idx_macro], X_MacroRev.iloc[test_idx_macro]
y_train_MacroRev, y_test_MacroRev = y_MacroRev.iloc[train_idx_macro], y_MacroRev.iloc[test_idx_macro]

In [219]:
# Train model based on the training data
model_MacroRev = LogisticRegression()
model_fit_MacroRev = model_MacroRev.fit(X_train_MacroRev, y_train_MacroRev)

c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [229]:
# 預測結果為1的機率
prob_MacroRev = predict_for_month(model_fit_MacroRev, TheWhole_rev, Varialble_MacroRev)
threshold_MacroRev = find_best_threshold(TheWhole_rev['real_direction'], prob_MacroRev) # 0.5092
#print(round(threshold_MacroRev, 4))

# 以閾值為界判斷預測值應為0或1
pred_MacroRev = predicted_labels(prob_MacroRev, threshold_MacroRev)

In [230]:
# 計算總經數值正確性
predicted_accuracy(TheWhole_rev['real_direction'], pred_MacroRev) # 0.62

0.62